In [1]:
!ollama pull orca-mini

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest 
pulling 66002b78c70a... 100% ▕████████████████▏ 2.0 GB                         
pulling dd90d0f2b7ee... 100% ▕████████████████▏   95 B                         
pulling 93ca9b3d83dc... 100% ▕████████████████▏   89 B                         
pulling 33eb43a1488d... 100% ▕████████████████▏   52 B                         
pulling fd52b10ee3ee... 100% ▕████████████████▏  455 B                         
verifying sha256 digest ⠋ pulling manifest 
pulling 66002b78c70a... 100% ▕████████████████▏ 2.0 GB                         
pulling dd90d0f2b7ee... 100% ▕████████████████▏   95 B                         
pulling 93ca9b3d83dc... 100% ▕████████████████▏   89 B                         
pulling 33eb43a1488d... 100% ▕████████████████▏   52 B                         
pulling fd52b10ee3ee... 100% ▕████████████████▏  455 B            

In [2]:
!pip install langchain
!pip install pymupdf
!pip install huggingface-hub
!pip install faiss-cpu
!pip install sentence-transformers

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [3]:
# Importing the necessary packages
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
import textwrap

from langchain.llms import Ollama
from langchain import PromptTemplate

/Users/meheksawhney/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [4]:
# This will load the PDF file
def load_pdf_data(file_path):
    # Creating a PyMuPDFLoader object with file_path
    loader = PyMuPDFLoader(file_path=file_path)
    
    # loading the PDF file
    docs = loader.load()
    
    # returning the loaded document
    return docs

In [5]:
docs = load_pdf_data(file_path="/Users/meheksawhney/Desktop/UI/attached_documents/quality-assurance-code-of-practice-approach-to-code-development.pdf")
print(docs)

[Document(page_content='Page 1 of 5 \n \n \n \n \n \nQUALITY ASSURANCE CODE OF PRACTICE: \nAPPROACH TO CODE DEVELOPMENT \n1. \nINTRODUCTION ..................................................................................................................... 1 \n2. \nTYPES OF REVISION ............................................................................................................. 1 \n3. \nREVISION PROCESS ............................................................................................................. 2 \n4. \nEQUALITY AND DIVERSITY ................................................................................................... 3 \n5. \nMONITORING AND REVIEW .................................................................................................. 3 \n1. \nINTRODUCTION \n \n \nThe University’s Quality Assurance Code of Practice (QA CoP) is a constantly evolving document. \nGood practice requires a regular review of policies and procedures. Rigorous at

In [6]:
# Responsible for splitting the documents into several chunks
def split_docs(documents, chunk_size=1000, chunk_overlap=20):
    
    # Initializing the RecursiveCharacterTextSplitter with
    # chunk_size and chunk_overlap
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    
    # Splitting the documents into chunks
    chunks = text_splitter.split_documents(documents=documents)
    
    # returning the document chunks
    return chunks

In [7]:
documents = split_docs(documents=docs)
print(len(documents))

14


In [8]:
# function for loading the embedding model
def load_embedding_model(model_path, normalize_embedding=True):
    return HuggingFaceEmbeddings(
        model_name=model_path,
        model_kwargs={'device':'cpu'}, # here we will run the model with CPU only
        encode_kwargs = {
            'normalize_embeddings': normalize_embedding # keep True to compute cosine similarity
        }
    )

In [9]:
# Function for creating embeddings using FAISS
def create_embeddings(chunks, embedding_model):
    # Creating the embeddings using FAISS
    vectorstore = FAISS.from_documents(chunks, embedding_model)
    
    # returning the vectorstore
    return vectorstore

In [10]:
# Creating the chain for Question Answering
def load_qa_chain(retriever, llm):
    return RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever, # here we are using the vectorstore as a retriever
        chain_type="stuff",
        return_source_documents=True, # including source documents in output
    )

In [11]:
# Prettifying the response
def get_response(query, chain):
    # Getting response from chain
    response = chain({'query': query})
    print(len(response['source_documents']))
    return response


In [12]:
# Loading orca-mini from Ollama
llm = Ollama(model="orca-mini", temperature=0)

# Loading the Embedding Model
embed = load_embedding_model(model_path="all-MiniLM-L6-v2")

/Users/meheksawhney/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
# creating vectorstore
vectorstore = create_embeddings(documents, embed)

# converting vectorstore to a retriever
retriever = vectorstore.as_retriever()

# Creating the chain
chain = load_qa_chain(retriever, llm)

In [14]:
get_response("what are the types of revision mentioned in the document?", chain)

/Users/meheksawhney/Library/Python/3.9/lib/python/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


4


{'query': 'what are the types of revision mentioned in the document?',
 'result': ' The types of revision mentioned in the document are:\n\n1. Generic changes: These involve updating statements with new University terminology, changes in organizational structures or titles, or incorporating the consequences of other broad changes already approved by senior committees including Senate.\n\n2. Incremental procedural and substantive changes: These are typically proposed on an ad hoc basis to address issues that have emerged, or to take forward enhancement opportunities which have been identified, through operation of the University’s routine quality management processes.\n\n3. Major reviews: These involve a major rewrite of a statement, or the creation of a new one, and require the approval of key principles by Senate. Such reviews will often be driven by University strategic priorities, and/or national developments.',
 'source_documents': [Document(page_content='this context. \n \nThe aim